# Synaptome parameterization example

Adds physiological parameter values drawn from pathway-specific model distributions to all synapses in a synaptome (i.e., a circuit of scale "single").

In [1]:
import obi_one as obi
from entitysdk import Client, ProjectContext, models
from obi_auth import get_token
from obi_notebook import get_projects, get_entities, get_environment

In [2]:
# environment= get_environment.get_environment()
environment = "staging"
token = get_token(environment=environment)

project_context = get_projects.get_projects(token=token, env=environment)

Dropdown(description='Select:', options=(('My first project', {'id': '7a411785-6895-4839-aaa2-d9f76e09875a', '…

In [3]:
client = Client(project_context=project_context, token_manager=token, environment=environment)

In [ ]:
synaptomes = get_entities.get_entities(
    entity_type="circuit",
    exclude_scales=["pair", "small", "microcircuit", "region", "system", "whole_brain"],
    add_columns=["build_category"],
    token=token,
    env=environment,
    project_context=project_context,
    multi_select=False,
    result=[],
)

In [26]:
# # Manual search for EM synaptome (i.e., Circuit with scale "single" and build category "em_reconstruction")
# synaptomes = client.search_entity(entity_type=models.Circuit, query={"scale": "single", "build_category": "em_reconstruction"}).all()
# assert len(synaptomes) > 0, "ERROR: No synaptomes built from EM found!"

# for syn in synaptomes:
#     print(f"{syn.name} (ID {syn.id})")

In [4]:
# Define CircuitFromID by copy-pasting one of the above IDs
# synaptome_id = "<PASTE-ID-HERE>"
synaptome_id = "a1afc2bd-45a2-4e4b-9cca-bdf4e8c0d48a"
# synaptome_id = synaptomes[0]
synaptome_from_id = obi.CircuitFromID(id_str=synaptome_id)

In [5]:
config_init = obi.SynapseParameterizationSingleConfig.Initialize(
    synaptome=synaptome_from_id,
    pathway_param_dict={},
    overwrite_if_exists=False,
)
task_config = obi.SynapseParameterizationSingleConfig(initialize=config_init)

In [6]:
task = obi.SynapseParameterizationTask(config=task_config)

In [7]:
task.execute(db_client=client, entity_cache=True)

[2025-11-21 16:26:17,759] INFO: Running synapse parameterization...
[2025-11-21 16:26:17,760] ERROR: Not yet implemented!
